In [1]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [2]:
from wefe.query import Query
from wefe.metrics import WEAT, RNSB, RND, MAC, ECT
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets.datasets import load_weat, fetch_eds, fetch_debias_multiclass, fetch_debiaswe, load_bingliu
from wefe.utils import run_queries, plot_queries_results, create_ranking, plot_ranking, plot_ranking_correlations
from plotly.subplots import make_subplots
import gensim
import numpy as np

In [3]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"

In [4]:
def get_google_news_embeddings(filename,binary=False):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=binary, no_header=True)
    return w2v_model

In [5]:
fasttext_wiki_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:40:18.310513', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [6]:
fasttext_wiki_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:41:41.447682', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [7]:
fasttext_cc_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1999995, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:44:01.293546', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [8]:
fasttext_cc_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2000000, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:46:19.371275', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [9]:
debiased_w2v_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"/debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin",binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-19T13:46:32.010448', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-gli

#### WEAT Effect Size: represents a normalized measure that quantifies how far apart the two distributions of association between targets and attributes are.

#### WEAT score:  Its objective is to quantify the strength of association of both pair of sets through a permutation test. The score being positive and higher than one indicates that word2vec exhibits a moderately strong relationship between men's names and careers and women's names and family.

In [10]:
# load the weat word sets
WEAT_wordsets = load_weat()
RND_wordsets = fetch_eds()
sentiments_wordsets = load_bingliu()
debias_multiclass_wordsets = fetch_debias_multiclass()

In [11]:
fasttext_wiki_model = WordEmbeddingModel(fasttext_wiki_model, 'fasttext_wiki')
fasttext_wiki_subwords_model = WordEmbeddingModel(fasttext_wiki_subwords_model, 'fasttext_wiki_subwords')
fasttext_cc_model = WordEmbeddingModel(fasttext_cc_model, 'fasttext_cc')
fasttext_cc_subwords_model = WordEmbeddingModel(fasttext_cc_subwords_model, 'fasttext_cc_subwords')
debiased_w2v_model = WordEmbeddingModel(debiased_w2v_model, 'debiased_w2v')

In [12]:
models = [fasttext_wiki_model, fasttext_wiki_subwords_model, fasttext_cc_model, fasttext_cc_subwords_model, debiased_w2v_model]

## Query ranking

In [13]:
# Create gender queries
gender_query_1 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']],
                       [WEAT_wordsets['career'], WEAT_wordsets['family']],
                       ['Male terms', 'Female terms'], ['Career', 'Family'])
gender_query_2 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']],
                       [WEAT_wordsets['science'], WEAT_wordsets['arts']],
                       ['Male terms', 'Female terms'], ['Science', 'Arts'])
gender_query_3 = Query([WEAT_wordsets['male_terms'], WEAT_wordsets['female_terms']],
                       [WEAT_wordsets['math'], WEAT_wordsets['arts_2']],
                       ['Male terms', 'Female terms'], ['Math', 'Arts'])

gender_query_4 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
    RND_wordsets['adjectives_intelligence'],
    RND_wordsets['adjectives_appearance']
], ['Male terms', 'Female terms'], ['Intelligence', 'Appearence'])

gender_query_5 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
    RND_wordsets['adjectives_intelligence'],
    RND_wordsets['adjectives_sensitive']
], ['Male terms', 'Female terms'], ['Intelligence', 'Sensitive'])


gender_sent_1 = Query(
    [RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
        sentiments_wordsets['positive_words'],
        sentiments_wordsets['negative_words']
    ], ['Male terms', 'Female terms'], ['Positive words', 'Negative words'])

gender_role_1 = Query(
    [RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
        debias_multiclass_wordsets['male_roles'],
        debias_multiclass_wordsets['female_roles']
    ], ['Male terms', 'Female terms'], ['Man Roles', 'Woman Roles'])

gender_queries = [gender_query_1, gender_query_2, gender_query_3, 
                  gender_query_4, gender_query_5, gender_role_1, gender_sent_1]

In [14]:
# Run the queries WEAT
WEAT_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    return_only_aggregation=True,
    queries_set_name='Gender Queries')


# Run the queries using RNSB
RNSB_gender_results = run_queries(RNSB,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                return_only_aggregation=True,
                                queries_set_name='Gender Queries')




In [15]:
WEAT_gender_results

,WEAT: Gender Queries average of abs values score
model_name,
fasttext_wiki,0.464644
fasttext_wiki_subwords,0.429728
fasttext_cc,0.592093
fasttext_cc_subwords,0.434877
debiased_w2v,0.079471


In [16]:
RNSB_gender_results

,RNSB: Gender Queries average of abs values score
model_name,
fasttext_wiki,0.026422
fasttext_wiki_subwords,0.003325
fasttext_cc,0.042487
fasttext_cc_subwords,0.013540
debiased_w2v,0.006403


In [17]:
# Run the queries using RND
RND_gender_results = run_queries(RND,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                generate_subqueries=True,
                                 return_only_aggregation=True,
                                queries_set_name='Gender Queries')




In [18]:
RND_gender_results

,RND: Gender Queries average of abs values score
model_name,
fasttext_wiki,0.038104
fasttext_wiki_subwords,0.029283
fasttext_cc,0.078313
fasttext_cc_subwords,0.048332
debiased_w2v,0.003118


In [19]:
# Run the queries using ECT
ECT_gender_results = run_queries(ECT,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                 return_only_aggregation=True,
                                generate_subqueries=True,
                                queries_set_name='Gender Queries')

In [20]:
ECT_gender_results

,ECT: Gender Queries average of abs values score
model_name,
fasttext_wiki,0.841350
fasttext_wiki_subwords,0.880892
fasttext_cc,0.707511
fasttext_cc_subwords,0.847711
debiased_w2v,0.981064
